In [ ]:
from sexp_repr import loads, dumps, validate, tree, path
from pprint import pprint
from src.shared.model import Node, Scalar
from src.shared.parser import Lexer, Parser
from src.sexp_schema.interpreter import SchemaNode, Interpreter
from src.sexp_schema.validator import Validator
from src.visualizer.cli import TreeRenderer

### Модель представления данных
1. `Scalar`

In [14]:
scalar_1 = Scalar(True)
scalar_2 = Scalar(10)
scalar_3 = Scalar("String")

for i in [scalar_1, scalar_2, scalar_3]:
    print(f"{repr(i)}, value: {i.value}")

Scalar(True), value: True
Scalar(10), value: 10
Scalar(String), value: String


2. `Node`

In [15]:
node_1 = Node(name="Ivan", attrs={"age": Scalar(5)}, scalar=Scalar("Son"))
node_2 = Node(name="Petr", attrs={"age": Scalar(25)}, children=[node_1])
print(node_1)
pprint(repr(node_1))
print()
print(node_2)
pprint(repr(node_2))
print()
print(node_1.to_sexp())

(Ivan (:age 5) "Son")
"Node(name='Ivan', attrs={'age': Scalar(5)}, children=[], value=Scalar(Son))"

(Petr (:age 25) (Ivan (:age 5) "Son"))
("Node(name='Petr', attrs={'age': Scalar(25)}, children=[Node(name='Ivan', "
 "attrs={'age': Scalar(5)}, children=[], value=Scalar(Son))], value=None)")

(Ivan (:age 5) "Son")


### Лексер и Парсер
1. `Lexer`

In [16]:
sexp = "(Alice 30)"
lexer = Lexer(sexp)
tokens = lexer.tokenize()
tokens

[Token(type='LPAREN', value='(', pos=0),
 Token(type='SYMBOL', value='Alice', pos=1),
 Token(type='NUMBER', value='30', pos=7),
 Token(type='RPAREN', value=')', pos=9),
 Token(type='EOF', value='', pos=10)]

2. `Parser`

In [17]:
parser = Parser(tokens).parse()
parser

Node(name='Alice', attrs={}, children=[], value=Scalar(30))

### Представление схемы (Sexp Schema)
1. `SchemaNode`

In [18]:
schema = """
(schema
  (element
    (:name "book")
    (attrs
      (attr (:name "lang") (type "string") (required false)))
    (children
      (element
        (:name "title")
        (type "string")
        (required true))

      (element
        (:name "author")
        (type "string")
        (required true)
        (attrs
          (attr (:name "born") (type "number") (required false))))

      (element
        (:name "year")
        (type "number")
        (required true))

      (element
        (:name "tags")
        (children
          (element
            (:name "tag")
            (type "string")
            (min_occurs 0)
            (max_occurs "unbounded")))))))
"""

schema = loads(schema)  # API-метод для получения Node
schema_node = Interpreter(schema).interpret()

pprint(schema_node)

SchemaNode(name='book',
           value_type=None,
           required=False,
           min_occurs=1,
           max_occurs='unbounded',
           attrs={'lang': SchemaNode(name=Scalar(lang),
                                     value_type=Scalar(string),
                                     required=False,
                                     min_occurs=1,
                                     max_occurs='unbounded',
                                     attrs={},
                                     children=[])},
           children=[SchemaNode(name='title',
                                value_type=Scalar(string),
                                required=True,
                                min_occurs=1,
                                max_occurs='unbounded',
                                attrs={},
                                children=[]),
                     SchemaNode(name='author',
                                value_type=Scalar(string),
                             

2. `Валидация`

In [19]:
sexp = """
(book
  (:lang "ru")
  (title "Война и мир")
  (author (:born 1828) "Лев Толстой")
  (year 1869)
  (tags
    (tag "classic")
    (tag "novel")))
"""

sexp_node = loads(sexp)
validator = Validator(sexp_node, schema_node).validate()

print(f"Результат соответствия документа схеме: {validator}")

Результат соответствия документа схеме: True


### CLI визуализация
1. Обработка S-выражения в виде AST (класс Node) 


In [20]:
root_document: Node = Node(name="Students")
student_1 = Node(name="Ivan", attrs={"age": Scalar(30)}, scalar=Scalar("Tired"))
student_2 = Node(name="Maria", attrs={"age": Scalar(20)}, scalar=Scalar("Fresh"))
root_document.add_child(student_1, student_2)
renderer = TreeRenderer()
renderer.render(root_document)

Students
├── Ivan :age=30
│   └── Tired
└── Maria :age=20
    └── Fresh

### API
Доступные методы:
- loads (`str` > `Node`)
- dumps (`Node` > `str`)
- validate (schema validate)
- tree (s-exp tree vizualization)
- path (SPath)

In [ ]:
l = loads("(Alice 20)")
print(repr(l), type(l))  # `loads` возвращает `Node`
d = dumps(Node(name="Alice"))
print(d, type(d))  # `dumps` возвращает дерево в формате `str`
v = validate(schema_document=schema, document=sexp_node)
print(v)  # `True` если валидация прошла успешно
tree(
    """
(book
  (:lang "ru")
  (title "Война и мир")
  (author (:born 1828) "Лев Толстой")
  (year 1869)
  (tags
    (tag "classic")
    (tag "novel")))
"""
)  # Принимает как `str`, так и `Node`

path(sexp, "/book/title")  # Возвращает `Node`, если путь найден и условия совпадают

Node(name='Alice', attrs={}, children=[], value=Scalar(20)) <class 'src.shared.model.Node'>
(Alice) <class 'str'>
True


book :lang=ru
├── title
│   └── Война и мир
├── author :born=1828
│   └── Лев Толстой
├── year
│   └── 1869
└── tags
    ├── tag
    │   └── classic
    └── tag
        └── novel

Node(name='title', attrs={}, children=[], value=Scalar(Война и мир))